# 4 Generating the Bank for the MicroTasks

To generate the bank for the microtasks I will use an API for an LLM.
The output will be two questions for each core course of each programme.

Basically I will create the perfect prompt that will use the columns of the df_courses to generate the microstasks. 

We use two prompts: broad + disambiguaition (Kenneth Style)

In [1]:
from pathlib import Path
import pandas as pd
#!pip install --upgrade openai
import os, re
from openai import OpenAI
import json
import numpy as np
from tqdm import tqdm  # optional progress bar, pip install tqdm


## 1 Load the data and filter for max 2 courses

In [2]:
# load the csv file about the courses forwhih we have to gen the tasks
silver = Path("../data_programmes_courses/silver")

df_courses_tasks = pd.read_csv(silver / "df_courses_tasks_silver.csv", encoding="utf-8-sig")
print("The shape of the courses tasks dataframe is:", df_courses_tasks.shape)

# keep only first two courses from each programme
df_courses_tasks = df_courses_tasks.groupby("programme_title").head(2).reset_index(drop=True)
print("After keeping only first two courses from each programme the shape is:", df_courses_tasks.shape)

The shape of the courses tasks dataframe is: (36, 21)
After keeping only first two courses from each programme the shape is: (28, 21)


## 2. Set up OpenAI client 

In [3]:

key_path = Path("../data_bank_microtasks") / "api_key.txt"

# Read the key and strip spaces and newlines
api_key = key_path.read_text(encoding="utf8").strip()

# Create the client using this key
client = OpenAI(api_key=api_key)

models = client.models.list()
#for m in models.data:
#    print(m.id)

model_gpt = "gpt-4.1-mini"  



## 3. Filtering the coding and math courses

In [5]:
# here we define manually which courses should count as python coding courses
# we can extend this set later if we want to add more courses
CODING_COURSES_PYTHON = {
    "Introduction to Programming",
    "Computer Programming",
    "Introduction to Business Analytics",  # we can remove this if it does not use python in practice
}

# here we define which courses are math related for puzzle tasks
MATH_PUZZLE_COURSES = {
    "Calculus and Analysis I",
    "Calculus 1",
    "Single Variable Calculus",
    "Basic Concepts in Mathematics",
    "Logic and Sets for CS",
    "Quantitative Research Methods I"
}

# here we add boolean flags to the dataframe
df_courses_tasks["uses_python"] = df_courses_tasks["course_name"].isin(CODING_COURSES_PYTHON)
df_courses_tasks["math_puzzle"] = df_courses_tasks["course_name"].isin(MATH_PUZZLE_COURSES)

print("Courses tagged as python:")
print(df_courses_tasks.loc[df_courses_tasks["uses_python"], "course_name"].unique())

print("\nCourses tagged as math for puzzle:")
print(df_courses_tasks.loc[df_courses_tasks["math_puzzle"], "course_name"].unique())


Courses tagged as python:
['Introduction to Programming' 'Introduction to Business Analytics'
 'Computer Programming']

Courses tagged as math for puzzle:
['Calculus and Analysis I' 'Calculus 1' 'Logic and Sets for CS'
 'Quantitative Research Methods I' 'Basic Concepts in Mathematics'
 'Single Variable Calculus']


In [6]:
def course_uses_python(row: pd.Series) -> bool:
    """
    With this function we decide if a course should receive a codeorder challenge.
    We use the manual flag uses_python that we added above.
    """
    return bool(row.get("uses_python", False))


In [7]:
# here we list all programme names, we still use this later for printing and merging
programmes = sorted(df_courses_tasks["programme_title"].unique())
print("Number of programmes:", len(programmes))
print("First few programmes:", programmes[:5])

def build_course_context(row: pd.Series) -> str:
    """
    With this function we build a short text snippet that describes one course.
    We anchor the challenge at course level instead of programme level.

    Input:
        row: one row of df_courses_tasks

    Output:
        context: string that describes the programme and this specific course
    """
    lines = []

    prog_name = row.get("programme_title", "")
    course_name = row.get("course_name", "")
    course_obj = row.get("course_objective", "")
    course_cont = row.get("course_content", "")

    if isinstance(prog_name, str) and prog_name.strip():
        lines.append(f"Programme: {prog_name.strip()}")

    if isinstance(course_name, str) and course_name.strip():
        lines.append(f"Course: {course_name.strip()}")

    if isinstance(course_obj, str) and course_obj.strip():
        lines.append(f"Objectives: {course_obj.strip()}")

    if isinstance(course_cont, str) and course_cont.strip():
        # here we keep a short snippet to control prompt length
        snippet = course_cont.strip()[:400]
        lines.append(f"Content snippet: {snippet}")

    context = "\n".join(lines)
    return context

# here we test the course context builder on one random row
sample_row = df_courses_tasks.sample(1, random_state=42).iloc[0]
print("Sample course context:")
print(build_course_context(sample_row))



Number of programmes: 14
First few programmes: ['Ancient Studies', 'Biomedical Sciences', 'Business Analytics', 'Communication and Information Studies', 'Computer Science']
Sample course context:
Programme: Literature and Society
Course: Creative Writing I
Objectives: Als je deze cursus hebt gevolgd kan je:
basale narratieve structuren en schrijftechnieken op het gebied van onder meer stijl, plot, scènes en personages, zoals o.a. beschreven en toegelicht door James Wood in Hoe fictie werkt, onderscheiden en toepassen;
aan de hand van aangereikte literaire teksten korte creatieve teksten schrijven;
zaken als het vertelperspectief en de vrije indirecte rede zelf op creatieve wijze invoegen en uitwerken in een afgerond eigen verhaal;
op respectvolle wijze feedback geven op het creatieve werk van medestudenten;
eigen teksten herschrijven aan de hand van de ontvangen feedback.
Content snippet: In het college Creatief schrijven verdiep je je in literaire technieken en leer je hoe je een verh

In [8]:
def build_aptitude_prompt(programme_name: str,
                          course_name: str,
                          course_context: str,
                          task_type: str) -> str:
    """
    With this function we build the text that we send to the model
    to create one aptitude microchallenge for a specific course.
    task_type can be "classify", "fillblank", "puzzle", "codeorder", or "graph".
    """
    base = f"""
    You are creating an aptitude micro challenge for a high school student of 16-17 years old
    who is curious about the bachelor programme {programme_name}
    and especially about the course {course_name}.

    You receive a short context that summarises this single course.
    You must anchor the challenge in this course context.
    Do not invent domains that are unrelated to this course.
    Use course topics, methods, and quantities that make sense for this course to generate the stimulus.
    The question cannot be difficult and should not be too long. Must be related to the main topoics of 
    the course but use a bit of imagination. remenber that the student is 16-17 years old and the tasks are for 
    university orientation purposes. Make them very intresting and appealing to a curious high school student. The instructions must be clear.

    The task must test ability or reasoning, not personal preference.
    Use instructions such as "Sort these", "Choose the correct", "Complete the text",
    "Arrange the code lines", or "Select the correct part of a graph".

    General requirements:
    - tiny_learn must be a list of exactly three short bullet points.
    Each bullet explains one useful idea in simple language that fits this course.
    - hint must be one short sentence that nudges the student without giving the answer away.
    - signalType must always be "aptitude".
    - The text of the task must clearly feel related to the course {course_name}.
"""

    if task_type == "classify":
        specific = """
Task type: classify.

You must return a JSON object with these fields:
question_code: string, for example "L_AABAOHW115_classify_1"
type: "classify"
signalType: "aptitude"
question: short instruction, for example "Sort these concepts into qualitative or quantitative"
tiny_learn: list of exactly three strings
categories: list of category labels, for example ["Qualitative", "Quantitative"]
items: list of objects with fields:
    id: short id such as "a" or "b"
    text: short description of the item
    correctCategory: one of the category labels
hint: short sentence

The categories and items must make sense for the course described in the context.
"""

    elif task_type == "fillblank":
        specific = """
Task type: fillblank.

You must return a JSON object with these fields:
question_code: string, for example "L_AABAOHW115_fillblank_1"
type: "fillblank"
signalType: "aptitude"
question: short instruction, for example "Complete the text"
tiny_learn: list of exactly three strings
textWithBlanks: short text that contains markers {{0}}, {{1}}, etc
blanks: list of objects with fields:
    id: integer index such as 0 or 1
    correctWordId: id of the correct word from the words list
words: list of objects with fields:
    id: string, for example "sumerian"
    text: the word as it should appear in the text
hint: short sentence

The text must describe something that fits the course context.
"""

    elif task_type == "puzzle":
        specific = """
Task type: puzzle.

You must return a JSON object with these fields and structure:
question_code: string, for example "L_AABAOHW115_puzzlebalance_1"
type: "puzzle"
signalType: "aptitude"
question: short instruction, for example "If 2 triangles balance 3 circles, how many circles balance 4 triangles?"
tiny_learn: list of exactly three strings

puzzle: object that describes the puzzle, for example:
    {
      "variant": "balance",
      "left": [
        { "shape": "▲", "count": 2 }
      ],
      "right": [
        { "shape": "●", "count": 3 }
      ],
      "unknownSide": "right",
      "unknownShape": "●"
    }
or:
    {
      "variant": "pattern",
      "sequence": ["1","1","2","3","5","8","?"]
    }

options: list of objects with fields:
    id: short letter id, "a", "b", "c", "d"
    value: the option value, for example a number or a short string
correctAnswer: id of the correct option, for example "c"
hint: short sentence

The puzzle must be solvable using proportional reasoning, pattern recognition,
or basic logic at high school level, and must be clearly related to the math
topics of the course context.
"""

    elif task_type == "codeorder":
        specific = """
Task type: codeorder.

You must return a JSON object with these fields:
question_code: string, for example "L_AABAOHW115_codeorder_5"
type: "codeorder"
signalType: "aptitude"
question: short instruction, for example "Arrange the code lines to compute the correct average grade"
tiny_learn: list of exactly three strings
language: must be "python"
description: short description of what the code should do in the context of this course
lines: list of objects with fields:
    id: short id such as "1", "2", "3"
    code: one line of python code as a string
    correctPosition: integer that gives the correct position in the final order
expectedOutput: short string that describes what the code prints or returns when correctly ordered
hint: short sentence

The code must be short and readable for a motivated high school student.
It must connect to a context that makes sense for this course.
The language must be python. Only python even if in the description another language is mentioned. ANd must be very simple.
"""

    elif task_type == "graph":
        specific = """
Task type: graph.

You must return a JSON object with these fields and structure:
question_code: string, for example "math-graph-001"
type: "graph"
signalType: "aptitude"
question: short instruction, for example
    "Looking at this graph of weekly study hours, which week showed the biggest increase?"
tiny_learn: list of exactly three strings

graphData: object that describes a simple graph, for example:
    {
      "type": "line",
      "title": "Weekly Study Hours",
      "labels": ["Week 1", "Week 2", "Week 3", "Week 4", "Week 5"],
      "values": [5, 7, 8, 14, 15],
      "yAxisLabel": "Hours"
    }
or:
    {
      "type": "bar",
      "title": "Student Success Rates in Academic Skills",
      "labels": ["Research Skills", "Argumentation", "Abstraction"],
      "values": [75, 85, 70]
    }

clickableRegions: list of objects with fields:
    id: short id such as "w1", "w2", "r1"
    label: label for the region or interval
    dataIndex: optional integer index that points to a position in the values list,
               used for line graphs when an interval spans two points

correctRegion: id of the correct region from clickableRegions
hint: short sentence

The graph must be simple and use quantities that match the course context,
for example exam scores, counts of artifacts, sample sizes, or weekly study hours.
"""

    else:
        raise ValueError(f"Unsupported task_type: {task_type}")

    context_block = f"""
Course context:
{course_context}

Output format:
Return a single valid JSON object and nothing else.
"""

    return base + specific + context_block


## 5. Generate one task per type per course, with question_code based on course code

In [10]:
def generate_aptitude_task_for_course(programme_name: str,
                                      course_name: str,
                                      course_context: str,
                                      task_type: str) -> dict:
    """
    With this function we call the model one time and return one aptitude task
    parsed as a Python dict for a specific course.
    """
    prompt = build_aptitude_prompt(
        programme_name=programme_name,
        course_name=course_name,
        course_context=course_context,
        task_type=task_type,
    )

    response = client.responses.create(
        model=model_gpt,
        input=prompt,
        temperature=0.7,
    )

    raw = response.output[0].content[0].text.strip()

    start = raw.find("{")
    end = raw.rfind("}") + 1

    if start == -1 or end <= start:
        raise ValueError("We did not find a JSON object in the model output.")

    json_str = raw[start:end]
    task = json.loads(json_str)

    # we enforce general fields from our side
    task["signalType"] = "aptitude"
    task["type"] = task_type
    task["programme_title"] = programme_name
    task["course_name"] = course_name

    # we normalise tiny_learn to exactly three bullets
    tiny = task.get("tiny_learn", [])
    if not isinstance(tiny, list):
        tiny = [str(tiny)]
    if len(tiny) > 3:
        tiny = tiny[:3]
    while len(tiny) < 3:
        tiny.append("Extra note about the concept.")
    task["tiny_learn"] = tiny

    # we strongly enforce language for codeorder
    if task_type == "codeorder":
        task["language"] = "python"

    return task


Now we add a helper to build question_code in the format you want.

In [11]:
# here we map each task type to a stable index, so codes are consistent
QUESTION_IDX_BY_TYPE = {
    "classify": 1,
    "fillblank": 2,
    "puzzle": 3,
    "graph": 4,
    "codeorder": 5,
}

def build_question_code(row: pd.Series, task_type: str) -> str:
    """
    With this function we build a question_code like L_AABAOHW115_fillblank_1
    using the course code and the task type.

    We expect a column called 'code' with values such as 'L_AABAOHW115'.
    If that is missing, we fall back to a simple slug from the course name.
    """
    course_code = row.get("code", "")
    if not isinstance(course_code, str) or not course_code.strip():
        # here we take a simple fallback based on course name
        name = str(row.get("course_name", "course")).strip()
        # we remove spaces and punctuation for a simple slug
        slug = re.sub(r"[^A-Za-z0-9]", "", name)
        course_code = slug or "COURSE"

    idx = QUESTION_IDX_BY_TYPE.get(task_type, 0)
    return f"{course_code}_{task_type}_{idx}"


## 6. Testing before the loop

In [13]:
test_programme = "Computer Science"  # change this to the programme we want

# Here we filter the dataframe to keep only rows for this programme
df_courses_test = df_courses_tasks[
    df_courses_tasks["programme_title"] == test_programme
].copy()

# Here we optionally keep only two core courses for this test
df_courses_test = (
    df_courses_test
    .groupby("programme_title", as_index=False)
    .head(2)
    .reset_index(drop=True)
)

print("After keeping two courses per programme for test:")
print(df_courses_test[["programme_title", "course_name", "code"]])

# Here we define which base task types we use for every course
BASE_TASK_TYPES = ["classify", "fillblank", "graph"]

# Here we keep a small bank only for the test programme
aptitude_bank_test: dict[str, dict] = {}

for _, row in tqdm(
    df_courses_test.iterrows(),
    total=len(df_courses_test),
    desc=f"Generating aptitude tasks for {test_programme}"
):
    prog = row["programme_title"]
    course_name = row["course_name"]
    ctx = build_course_context(row)

    # Here we decide the task types for this course
    task_types_for_course = list(BASE_TASK_TYPES)

    # Puzzle only for math related courses
    if bool(row.get("math_puzzle", False)):
        task_types_for_course.append("puzzle")

    # Codeorder only for python coding courses
    if course_uses_python(row):
        task_types_for_course.append("codeorder")

    tasks_for_this_course = []

    for task_type in task_types_for_course:
        try:
            task = generate_aptitude_task_for_course(
                programme_name=prog,
                course_name=course_name,
                course_context=ctx,
                task_type=task_type,
            )

            # Here we build a stable question code for this course and type
            task["question_code"] = build_question_code(row, task_type)

            tasks_for_this_course.append(task)
        except Exception as e:
            print(f"Problem for course {course_name} task {task_type}: {e}")

    if not tasks_for_this_course:
        continue

    if prog not in aptitude_bank_test:
        aptitude_bank_test[prog] = {"aptitude": []}
    aptitude_bank_test[prog]["aptitude"].extend(tasks_for_this_course)

print("Done for test programme:", test_programme)

After keeping two courses per programme for test:
    programme_title            course_name      code
0  Computer Science   Computer Programming  XB_40011
1  Computer Science  Logic and Sets for CS   XB_0086


Generating aptitude tasks for Computer Science: 100%|██████████| 2/2 [00:45<00:00, 22.51s/it]

Done for test programme: Computer Science


Run th loop that genrates the JSON file

In [14]:
from tqdm import tqdm

# base task types that any course can receive
BASE_TASK_TYPES = ["classify", "fillblank", "graph"]

aptitude_bank: dict[str, dict] = {}

for _, row in tqdm(
    df_courses_tasks.iterrows(),
    total=len(df_courses_tasks),
    desc="Generating aptitude microchallenges per course"
):
    prog = row["programme_title"]
    course_name = row["course_name"]
    ctx = build_course_context(row)

    # here we decide which task types we want for this course
    task_types_for_course = list(BASE_TASK_TYPES)

    # puzzle only for math related courses
    if bool(row.get("math_puzzle", False)):
        task_types_for_course.append("puzzle")

    # codeorder only for python coding courses
    if course_uses_python(row):
        task_types_for_course.append("codeorder")

    tasks_for_this_course = []

    for task_type in task_types_for_course:
        try:
            task = generate_aptitude_task_for_course(
                programme_name=prog,
                course_name=course_name,
                course_context=ctx,
                task_type=task_type,
            )
            # here we override question_code so it follows the L_AABAOHW115_fillblank_1 pattern
            task["question_code"] = build_question_code(row, task_type)
            tasks_for_this_course.append(task)
        except Exception as e:
            print(f"Problem for course {course_name} in programme {prog} task type {task_type}: {e}")

    if not tasks_for_this_course:
        continue

    # here we store the aptitude tasks grouped by programme
    if prog not in aptitude_bank:
        aptitude_bank[prog] = {"aptitude": []}
    aptitude_bank[prog]["aptitude"].extend(tasks_for_this_course)

# quick sample to inspect
sample_prog = programmes[0]
print("Example programme:", sample_prog)
print("Number of aptitude tasks for this programme:",
      len(aptitude_bank.get(sample_prog, {}).get("aptitude", [])))
if aptitude_bank.get(sample_prog, {}).get("aptitude"):
    print(json.dumps(aptitude_bank[sample_prog]["aptitude"][:2], indent=2, ensure_ascii=False))


Generating aptitude microchallenges per course:  82%|████████▏ | 23/28 [07:56<01:40, 20.16s/it]

Problem for course Single Variable Calculus in programme Mathematics task type graph: Invalid \escape: line 5 column 53 (char 138)


Generating aptitude microchallenges per course: 100%|██████████| 28/28 [09:23<00:00, 20.11s/it]

Example programme: Ancient Studies
Number of aptitude tasks for this programme: 6
[
  {
    "question_code": "L_AABAOHW115_classify_1",
    "type": "classify",
    "signalType": "aptitude",
    "question": "Sort these ancient objects according to whether they primarily belong to the sacred, domestic, or funerary context.",
    "tiny_learn": [
      "Ancient objects can be grouped by the role they played in daily life or rituals.",
      "Understanding the context of an object helps us learn about the people who used it.",
      "Objects from sacred, domestic, or funerary contexts often tell different historical stories."
    ],
    "categories": [
      "Sacred",
      "Domestic",
      "Funerary"
    ],
    "items": [
      {
        "id": "a",
        "text": "Bronze statue of a deity",
        "correctCategory": "Sacred"
      },
      {
        "id": "b",
        "text": "Clay cooking pot",
        "correctCategory": "Domestic"
      },
      {
        "id": "c",
        "text": "F

Saving the challenges

In [15]:
data_dir = Path("../data_bank_microtasks")
full_bank_path = data_dir / "microchallenges_bank_aptitude.json"

with open(full_bank_path, "w", encoding="utf-8") as f:
    json.dump(aptitude_bank, f, ensure_ascii=False, indent=2)

print("Saved full microtasks bank to:", full_bank_path)
print("Number of programmes in the full bank:", len(aptitude_bank))

# here we quickly check that one programme has both personality and aptitude if personality existed
sample_prog = programmes[0]
print("Sample programme:", sample_prog)
print("Keys for this programme:", aptitude_bank.get(sample_prog, {}).keys())

Saved full microtasks bank to: ..\data_bank_microtasks\microchallenges_bank_aptitude.json
Number of programmes in the full bank: 14
Sample programme: Ancient Studies
Keys for this programme: dict_keys(['aptitude'])


In [16]:
data_dir = Path("../data_bank_microtasks")
data_dir.mkdir(parents=True, exist_ok=True)

base_bank_path = data_dir / "microtasks_RIASEC.json"

if base_bank_path.exists():
    with open(base_bank_path, "r", encoding="utf-8") as f:
        microtasks_bank = json.load(f)
    print("Loaded existing microtasks_RIASEC.json")
else:
    microtasks_bank = {}
    print("No existing microtasks_RIASEC.json found, we start from an empty bank")

for prog, block in aptitude_bank.items():
    if prog not in microtasks_bank:
        microtasks_bank[prog] = {}
    microtasks_bank[prog]["aptitude"] = block["aptitude"]

full_bank_path = data_dir / "microtasks_bank_full.json"
with open(full_bank_path, "w", encoding="utf-8") as f:
    json.dump(microtasks_bank, f, ensure_ascii=False, indent=2)

print("Saved full microtasks bank to:", full_bank_path)
print("Number of programmes in the full bank:", len(microtasks_bank))


Loaded existing microtasks_RIASEC.json
Saved full microtasks bank to: ..\data_bank_microtasks\microtasks_bank_full.json
Number of programmes in the full bank: 14
